In [ ]:
import numpy as np

In [ ]:
class GridEnv:
  def __init__(self, grid, gamma, noise):
    self.grid = grid
    self.gamma = gamma
    self.noise = noise

    self.size = len(grid)

    self.states = [(i, j) for i in range (self.size) for j in range (self.size)]
    self.actions= [0, 1, 2, 3]

    self.P = {}

    for s in self.states:
      self.P[s] = {}
      if self.isValid(s) == False:
        continue

      for a in self.actions:
        self.P[s][a] = []
        next_state, reward = self.take_action(s, a)

        exit = (reward == 10 or reward == 0 or reward == -10 or self.isValid(next_state)== False)

        for i in range (4):
          if i == a:
            self.P[s][a].append((4/5, next_state, reward, exit))
          else:
            next_state, reward = self.take_action(i,s)
            self.P[s][a].append((1/15,next_state, reward, exit))


  def isValid(self, state):
    i, j = state
    return 0 <= i < self.size and 0 <= j < self.size and self.grid[i][j] != 'wall'

  def take_action(self, a, s):
    i, j = s

    if a == 0:
      j -=1
    elif a == 1:
      j +=1
    elif a == 2:
      i -=1
    elif a == 3:
      i +=1
    else:
      raise ValueError("Invalid action")

    next_state = (i, j)

    if self.isValid(next_state):
      x = next_state[0]
      y = next_state[1]

      value = self.grid[x][y]
      if (value == 'start' or value == 'wall'):
        reward = 0
      else:
        reward = self.grid[x][y]
    else:
      reward = self.grid[x][y]

    return next_state, reward

In [ ]:
def policy_evaluation(env, policy, gamma=0.9, theta=1e-6):
  num_states = len(env.states)
  n = env.size
  V = np.zeros((n, n))

  num_iterations = 0
  while True:
      delta = 0
      for i in range(n):
        for j in range(n):
          s = (i, j)
          if not env.isValid(s):
            continue
          v = V[i][j]
          a = policy[s]
          for prob, next_state, reward, _ in env.P[s][a]:
              v += prob * (reward + gamma * V[next_state[0]][next_state[0]])
          V[i][j] = v
          delta = max(delta, np.abs(v - V[i][j]))
      num_iterations += 1
      if delta < theta:
          break
  return V, num_iterations

In [ ]:
def find_policy(V):
  r, c = V.shape
  directions = np.zeros((r, c), dtype=int)

  for i in range(r):
    for j in range(c):
      value = V[i][j]
      max = value
      direction = -1

      neighbors = [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]

      for a, b in neighbors:
        if 0 <= a < r and 0 <= b < c and V[a, b] > max:
          max = V[a, b]
          direction = neighbors.index((a, b))

        directions[i, j] = direction

  return directions

In [ ]:
def value_iteration(env, gamma=0.9, theta=1e-6):

  num_states = len(env.states)
  num_actions = len(env.actions)
  n = env.size
  V = np.zeros(n, n)

  num_iterations = 0
  while True:
      delta = 0
      for i in range(n):
        for j in range(n):
          s = (i, j)
          if not env.isValid(s):
            continue
          v = V[i][j]
          q_values = np.zeros(num_actions)
          for a in env.actions:
              for prob, next_state, reward, _ in env.P[s][a]:
                  q_values[a] += prob * (reward + gamma * V[next_state[0]][next_state[1]])
          V[i][j] = max(q_values)
          delta = max(delta, abs(v - V[i][j]))
      num_iterations += 1
      if delta < theta:
          break
      optimal_policy = find_policy(V)
  return optimal_policy, num_iterations, V

In [ ]:
grid = [[0, 0, 0, 0, 0],
        [0, 'wall', 0, 0, 0],
        [0, 'wall', 1, 'wall', 10],
        ['start', 0, 0, 0, 0],
        [-10, -10, -10, -10, -10]]

gammas = [0.2, 0.5, 0.9]
noises = [0.2, 0.3, 0.5]

for gamma in gammas:
  for noise in noises:
    env = GridEnv(grid, gamma, noise)
    optimal_policy, num_iterations, V = value_iteration(env, gamma=gamma, theta=1e-6)
    print(num_iterations)
    print(f"For Gamma {gamma}, noise {noise} value function is {V}")

    n = len(grid)
    for i in range(n):
      for j in range(n):
        s = (i, j)
        if s in env.states:
          a = optimal_policy[s[0]][s[1]]
          if a == 0:
            print("left")
          elif a == 1:
            print("right")
          elif a == 2:
            print("Up")
          elif a == 3:
            print("Bottom")
        else:
          print(grid[i][j])